In [37]:
import numpy as np
import pandas as pd

import pymysql
import dotenv
import os
import pickle
import seaborn as sns
import matplotlib

from datetime import timedelta

import matplotlib.pyplot as plt

# Normalization/Standardization
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout, Conv1D, GRU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.saving import save_model

# 경고 무시 코드 추가
import warnings
warnings.filterwarnings('ignore')

def pltconfig_default() :
  sns.reset_defaults()
  %matplotlib inline

pltconfig_default()

matplotlib.rcParams

matplotlib.rcParams['font.family']

current_font_list = matplotlib.rcParams['font.family']

font_path = 'C:\\Windows\\Fonts\\batang.ttc'

kfont = matplotlib.font_manager.FontProperties(fname=font_path).get_name()

matplotlib.rcParams['font.family'] = [kfont] + current_font_list

In [38]:
St_NotEncode_data = pd.read_pickle("StandardScalar_final_data")

print(St_NotEncode_data.shape)

(17422, 9)


In [39]:
# Feature와 Label 분리하기 (습도, 일조시간, 전운량)
def Feature_Label(datafile) :
    X = datafile.iloc[:,[3,4,5]]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SNE_X, SNE_y = Feature_Label(St_NotEncode_data)
print(SNE_X.shape, SNE_y.shape)
SNE_X_train, SNE_X_test, SNE_y_train, SNE_y_test = train_test_split(SNE_X, SNE_y, test_size=0.2, random_state=10, shuffle=False)
print(SNE_X_train.shape, SNE_X_test.shape, SNE_y_train.shape, SNE_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 3) (17422,)
(13937, 3) (3485, 3) (13937,) (3485,)


In [40]:
# X 값 Window Dataset 구성
ds_x = tf.data.Dataset.from_tensor_slices(SNE_X_train).window(WINDOW_SIZE, stride=1, shift=1, drop_remainder=True)  # 여기서는 X값만 별도로 구성하므로 WINDOW_SIZE 값에 1을 더하지 않는다!
ds_x = ds_x.flat_map(lambda x : x.batch(WINDOW_SIZE))

In [41]:
# y 값 Window Dataset 구성
ds_y = tf.data.Dataset.from_tensor_slices(SNE_y_train[WINDOW_SIZE:])

In [42]:
train_data = tf.data.Dataset.zip((ds_x, ds_y)).batch(BATCH_SIZE)

In [43]:
for x, y in train_data.take(1) :
  print(x[:3])
  print()
  print(y[:3])

tf.Tensor(
[[[ 0.09445245 -0.66581843 -1.23511514]
  [ 0.4051263  -0.66581843 -1.23511514]
  [ 0.53827224 -0.66581843 -1.23511514]]

 [[ 0.4051263  -0.66581843 -1.23511514]
  [ 0.53827224 -0.66581843 -1.23511514]
  [ 0.71580015 -0.66581843 -1.23511514]]

 [[ 0.53827224 -0.66581843 -1.23511514]
  [ 0.71580015 -0.66581843 -1.23511514]
  [ 0.80456411 -0.66581843 -1.23511514]]], shape=(3, 3, 3), dtype=float64)

tf.Tensor([0. 0. 0.], shape=(3,), dtype=float64)


In [44]:
pd.concat([SNE_X_train, SNE_y_train], axis=1).head(10)

,3,4,5,8
0,0.094452,-0.665818,-1.235115,0.0
1,0.405126,-0.665818,-1.235115,0.0
2,0.538272,-0.665818,-1.235115,0.0
3,0.715800,-0.665818,-1.235115,0.0
4,0.804564,-0.665818,-1.235115,0.0
5,0.937710,-0.665818,-1.235115,0.0
6,0.893328,-0.665818,-1.235115,0.0
7,0.982092,0.549517,-1.235115,6000.0
8,0.538272,1.764852,0.769233,16000.0
9,0.005688,1.764852,-0.984572,24000.0


In [45]:
# 위 내용을 함수 형태로 만들고 싶은 경우

def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(x).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(y[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [46]:
train_data = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SNE_X_test, SNE_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [47]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 3), (28,)


In [48]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 3]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [49]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 3]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [50]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 3]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [51]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [52]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('new_multi_stne_lstm_weight_c3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('new_multi_stne_lstm_weight_ns_c3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('new_multi_stne_rnn_weight_c3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('new_multi_stne_rnn_weight_ns_c3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('new_multi_stne_gru_weight_c3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('new_multi_stne_gru_weight_ns_c3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [53]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500
    495/Unknown - 27s 10ms/step - loss: 7608.4946 - mse: 183180400.0000 - mae: 7608.8877
Epoch 1: val_loss improved from inf to 7594.19678, saving model to new_multi_stne_lstm_weight_c3.h5
498/498 [==============================] - 34s 24ms/step - loss: 7616.3223 - mse: 183387520.0000 - mae: 7616.7158 - val_loss: 7594.1968 - val_mse: 182409888.0000 - val_mae: 7594.6274 - lr: 0.0010
Epoch 2/500
494/498 [============================>.] - ETA: 0s - loss: 7537.4966 - mse: 180408720.0000 - mae: 7537.8813
Epoch 2: val_loss improved from 7594.19678 to 7515.09863, saving model to new_multi_stne_lstm_weight_c3.h5
498/498 [==============================] - 7s 13ms/step - loss: 7557.8481 - mse: 180904480.0000 - mae: 7558.2329 - val_loss: 7515.0986 - val_mse: 178839312.0000 - val_mae: 7515.4556 - lr: 0.0010
Epoch 3/500
495/498 [============================>.] - ETA: 0s - loss: 7456.7900 - mse: 176546272.0000 - mae: 7457.1582
Epoch 3: val_loss improved from 7515.09863 to 7435.64014, sav

In [54]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500
    490/Unknown - 8s 3ms/step - loss: 7622.6406 - mse: 183791776.0000 - mae: 7622.9971
Epoch 1: val_loss improved from inf to 7619.69531, saving model to new_multi_stne_rnn_weight_c3.h5
498/498 [==============================] - 11s 8ms/step - loss: 7625.5405 - mse: 183826288.0000 - mae: 7625.8975 - val_loss: 7619.6953 - val_mse: 183554768.0000 - val_mae: 7620.1299 - lr: 0.0010
Epoch 2/500
480/498 [===========================>..] - ETA: 0s - loss: 7554.7231 - mse: 181884464.0000 - mae: 7555.0859
Epoch 2: val_loss improved from 7619.69531 to 7600.71729, saving model to new_multi_stne_rnn_weight_c3.h5
498/498 [==============================] - 3s 6ms/step - loss: 7610.3516 - mse: 183173344.0000 - mae: 7610.7158 - val_loss: 7600.7173 - val_mse: 182702400.0000 - val_mae: 7601.0425 - lr: 0.0010
Epoch 3/500
492/498 [============================>.] - ETA: 0s - loss: 7570.8691 - mse: 181762272.0000 - mae: 7571.2290
Epoch 3: val_loss improved from 7600.71729 to 7586.73975, saving mo

In [55]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500
    496/Unknown - 8s 5ms/step - loss: 7624.8418 - mse: 183828800.0000 - mae: 7625.2168
Epoch 1: val_loss improved from inf to 7605.68848, saving model to new_multi_stne_gru_weight_c3.h5
498/498 [==============================] - 11s 11ms/step - loss: 7620.5962 - mse: 183608144.0000 - mae: 7620.9707 - val_loss: 7605.6885 - val_mse: 182915136.0000 - val_mae: 7606.0723 - lr: 0.0010
Epoch 2/500
490/498 [============================>.] - ETA: 0s - loss: 7569.7119 - mse: 181942288.0000 - mae: 7570.0815
Epoch 2: val_loss improved from 7605.68848 to 7563.77344, saving model to new_multi_stne_gru_weight_c3.h5
498/498 [==============================] - 4s 7ms/step - loss: 7579.3018 - mse: 181829712.0000 - mae: 7579.6738 - val_loss: 7563.7734 - val_mse: 180380608.0000 - val_mae: 7564.1870 - lr: 0.0010
Epoch 3/500
498/498 [==============================] - ETA: 0s - loss: 7509.7622 - mse: 178762048.0000 - mae: 7510.1328
Epoch 3: val_loss improved from 7563.77344 to 7463.20215, saving m

In [56]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 1067.066650390625
lstm_history Validation MSE: 9518834.0
lstm_history Validation MAE: 1067.400634765625
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 1078.3626708984375
lstm_history_ns Validation MSE: 9509067.0
lstm_history_ns Validation MAE: 1078.69775390625
-------------rnn_history-------------
rnn_history Validation Loss: 1342.8851318359375
rnn_history Validation MSE: 11795144.0
rnn_history Validation MAE: 1343.2628173828125
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 1130.990478515625
rnn_history_ns Validation MSE: 10294384.0
rnn_history_ns Validation MAE: 1131.3466796875
-------------gru_history-------------
gru_history Validation Loss: 1420.2073974609375
gru_history Validation MSE: 13199008.0
gru_history Validation MAE: 1420.5570068359375
-------------gru_history_ns-------------
gru_history_ns Validation Loss: 1220.2569580078125
gru_history_ns Validation 

In [57]:
lstm_save_path = f"./Models/lstm_model_stne_c3"
rnn_save_path = f"./Models/rnn_model_stne_c3"
gru_save_path = f"./Models/gru_model_stne_c3"

save_model(lstm_model, lstm_save_path, overwrite=True)
save_model(rnn_model, rnn_save_path, overwrite=True)
save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_c3\assets


INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_c3\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_c3\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_c3\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_c3\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_c3\assets


In [58]:
St_NotEncode_Basic_data = pd.read_pickle("Basic_StandardScalar_final_data")
  
print(St_NotEncode_Basic_data.shape)

(52578, 9)


In [59]:
# Feature와 Label 분리하기 (습도, 일조시간, 일사량, 전운량)
def Feature_Label(datafile) :
    X = datafile.iloc[:,[3,4,5,6]]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SB_X, SB_y = Feature_Label(St_NotEncode_Basic_data)
print(SNE_X.shape, SNE_y.shape)
SB_X_train, SB_X_test, SB_y_train, SB_y_test = train_test_split(SB_X, SB_y, test_size=0.2, random_state=10, shuffle=False)
print(SB_X_train.shape, SB_X_test.shape, SB_y_train.shape, SB_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 3) (17422,)
(42062, 4) (10516, 4) (42062,) (10516,)


In [60]:
def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(x).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(y[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [61]:
train_data = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SB_X_test, SB_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [62]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 4), (28,)


In [63]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 4]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [64]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 4]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [65]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 4]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [66]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [67]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('base_multi_stne_lstm_weight_c4.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('base_multi_stne_lstm_weight_ns_c4.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('base_multi_stne_rnn_weight_c4.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('base_multi_stne_rnn_weight_ns_c4.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('base_multi_stne_gru_weight_c4.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('base_multi_stne_gru_weight_ns_c4.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [68]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500
   1502/Unknown - 22s 9ms/step - loss: 1.9709 - mse: 21.3240 - mae: 2.3328
Epoch 1: val_loss improved from inf to 2.11466, saving model to base_multi_stne_lstm_weight_c4.h5
1503/1503 [==============================] - 25s 11ms/step - loss: 1.9709 - mse: 21.3230 - mae: 2.3327 - val_loss: 2.1147 - val_mse: 21.3098 - val_mae: 2.5101 - lr: 0.0010
Epoch 2/500
1499/1503 [============================>.] - ETA: 0s - loss: 1.3104 - mse: 9.3724 - mae: 1.6546
Epoch 2: val_loss improved from 2.11466 to 2.03732, saving model to base_multi_stne_lstm_weight_c4.h5
1503/1503 [==============================] - 14s 9ms/step - loss: 1.3121 - mse: 9.4111 - mae: 1.6564 - val_loss: 2.0373 - val_mse: 20.2499 - val_mae: 2.4237 - lr: 0.0010
Epoch 3/500
1498/1503 [============================>.] - ETA: 0s - loss: 1.2817 - mse: 9.1371 - mae: 1.6243
Epoch 3: val_loss did not improve from 2.03732
1503/1503 [==============================] - 14s 10ms/step - loss: 1.2839 - mse: 9.1572 - mae: 1.6266 - val_

In [69]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500
   1495/Unknown - 7s 3ms/step - loss: 2.1540 - mse: 26.1920 - mae: 2.5093
Epoch 1: val_loss improved from inf to 2.19272, saving model to base_multi_stne_rnn_weight_c4.h5
1503/1503 [==============================] - 8s 4ms/step - loss: 2.1543 - mse: 26.1985 - mae: 2.5097 - val_loss: 2.1927 - val_mse: 22.2660 - val_mae: 2.5835 - lr: 0.0010
Epoch 2/500
1495/1503 [============================>.] - ETA: 0s - loss: 1.3439 - mse: 9.8390 - mae: 1.6883
Epoch 2: val_loss improved from 2.19272 to 1.99480, saving model to base_multi_stne_rnn_weight_c4.h5
1503/1503 [==============================] - 6s 4ms/step - loss: 1.3458 - mse: 9.8553 - mae: 1.6904 - val_loss: 1.9948 - val_mse: 19.1653 - val_mae: 2.3739 - lr: 0.0010
Epoch 3/500
1486/1503 [============================>.] - ETA: 0s - loss: 1.2964 - mse: 9.2945 - mae: 1.6359
Epoch 3: val_loss did not improve from 1.99480
1503/1503 [==============================] - 6s 4ms/step - loss: 1.3018 - mse: 9.3973 - mae: 1.6416 - val_loss: 3.

In [70]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500
   1498/Unknown - 17s 6ms/step - loss: 1.9374 - mse: 21.2114 - mae: 2.2923
Epoch 1: val_loss improved from inf to 2.05985, saving model to base_multi_stne_gru_weight_c4.h5
1503/1503 [==============================] - 19s 7ms/step - loss: 1.9370 - mse: 21.1939 - mae: 2.2919 - val_loss: 2.0598 - val_mse: 19.8674 - val_mae: 2.4415 - lr: 0.0010
Epoch 2/500
1494/1503 [============================>.] - ETA: 0s - loss: 1.2897 - mse: 9.1806 - mae: 1.6307
Epoch 2: val_loss did not improve from 2.05985
1503/1503 [==============================] - 10s 7ms/step - loss: 1.2930 - mse: 9.2548 - mae: 1.6341 - val_loss: 2.3625 - val_mse: 20.7834 - val_mae: 2.7746 - lr: 0.0010
Epoch 3/500
1496/1503 [============================>.] - ETA: 0s - loss: 1.2593 - mse: 9.0012 - mae: 1.5942
Epoch 3: val_loss improved from 2.05985 to 1.98064, saving model to base_multi_stne_gru_weight_c4.h5
1503/1503 [==============================] - 10s 7ms/step - loss: 1.2617 - mse: 9.0210 - mae: 1.5967 - val_loss

In [71]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 1.7359941005706787
lstm_history Validation MSE: 16.42426872253418
lstm_history Validation MAE: 2.096876859664917
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 1.8730195760726929
lstm_history_ns Validation MSE: 18.03646469116211
lstm_history_ns Validation MAE: 2.2363853454589844
-------------rnn_history-------------
rnn_history Validation Loss: 1.7492930889129639
rnn_history Validation MSE: 16.273529052734375
rnn_history Validation MAE: 2.135190963745117
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 1.8690425157546997
rnn_history_ns Validation MSE: 17.682697296142578
rnn_history_ns Validation MAE: 2.228328227996826
-------------gru_history-------------
gru_history Validation Loss: 1.7354586124420166
gru_history Validation MSE: 15.914024353027344
gru_history Validation MAE: 2.0929148197174072
-------------gru_history_ns-------------
gru_history_ns Validation Loss: 

In [72]:
lstm_save_path = f"./Models/lstm_model_base_c4"
rnn_save_path = f"./Models/rnn_model_base_c4"
gru_save_path = f"./Models/gru_model_base_c4"

save_model(lstm_model, lstm_save_path, overwrite=True)
save_model(rnn_model, rnn_save_path, overwrite=True)
save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Models/lstm_model_base_c4\assets


INFO:tensorflow:Assets written to: ./Models/lstm_model_base_c4\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_base_c4\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_base_c4\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_base_c4\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_base_c4\assets
